In [ ]:
# Required packages / imports

import pandas as pd
import itertools as it
import math
import altair as alt
import os
import numpy as np
import seaborn as sns
import matplotlib.colors as colors
import glob
from skbio.stats import ordination
from scipy.stats import mannwhitneyu
from functools import lru_cache,reduce

alt.data_transformers.disable_max_rows()

os.makedirs('Output',exist_ok=True)

### Table of Contents

* [Definitions / Helper Functions](#defines)
* [Load Data](#loaddata)
* [Normalization Function](#normalize)
* Analysis
    * [Top Level Statistics](#toplevel)
    * [P-Values Top Level Statistics](#pvalues)
    * [Overview Top Taxa per Level](#toptaxa)
    * [Unmapped and Human Fraction](#human)
    * [AMR Gene Counts](#amr)
    * [Taxa Counts (Diversity)](#taxacount)
    * [Barplots Compositions](#barplots)
    * [Patient Journeys](#barplots_time)
    * [Bray-Curtis Based PCoA](#pcoa)
    * [Sampling Times Overview](#sampletimes)
    * [Zymo Std Analysis](#zymo)
    * [Eukaryotes Analysis](#eukaryotes)
    * [Crassphage Analysis](#crassphage)
    * [Population Level Analysis / Illumina](#strains)



# Definitions / Helper Functions <a class="anchor" id="defines"></a>

In [ ]:
def generatePrePostPairs(table : pd.DataFrame, patientcolumn: str = 'patientid' , timecolumn: str ='time' ):
    result = []
    counts_of_common_occurences = table.groupby([patientcolumn,timecolumn]).size().reset_index()
    for patient in counts_of_common_occurences[patientcolumn].unique():
        subtable = counts_of_common_occurences[counts_of_common_occurences[patientcolumn] == patient]
        for time1,time2 in it.combinations(subtable[timecolumn].unique(),2):
            if time1 < 0 and time2 > 0:
                result.append((patient,time1,time2))
            elif time1 > 0 and time2 < 0:
                result.append((patient,time2,time1))
    return result


#Helper Function that returns a preceding timepoint (if one exists) for each row in a table
def find_previous_sample(table,row):
    #print (row ['PatID'])
    timepoints_patient = list(table[table['PatID']==row['PatID']]['time'].unique())
    timepoints_filtered = [timepoint for timepoint in timepoints_patient if timepoint < row['time']]
    if len(timepoints_filtered) == 0:
        return None
    else:
        return sorted(timepoints_filtered)[-1]

#Helper function that recalculates some relative days and relabels some patients
def adjust_table(df):

    df.loc[(df['patientid'] == '999') & (df['time'] == 0), 'patientid'] = 'G1'
    df.loc[df['patientid'] == 'G1', 'time'] = -100
    df.loc[df['patientid'] == '132', 'patientid'] = '45'
    df.loc[df['patientid'] == '133', 'patientid'] = '14'
    df.loc[df['patientid'] == '134', 'patientid'] = '46'

def sort_samples(samples):
    return sorted(
        samples,
        key=lambda x : ('G' in x,
                        int(x.split('/')[0].replace('G','').split('.')[0]),
                        int(x.split('/')[0].replace('G','').split('.')[-1]),
                        int(x.split('/')[1]))
    )

#Constants, frequently used
CHORDATA = '7711'
FUNGI = '4751'
VIRUSES = '10239'
BACTERIA = '2'
ARCHAEA = '2157'
EUKARYOTA = '2759'
PROTOZOA = '1891100'
OFFSET_PAT_18 = 161

PAPER_SAMPLES = ['G1/-100','G2/-100','G3/-100','G4/-100','G5/-100','G6/-100',
             'G7/-100','G8/-100','G9/-100','G10/-100','G11/-100',
             '4/-7','7/-6','11/-2','12/-6','14/-7','15/-6','16/-5','17/-7','19/-2',
             '21/-9','24/-10','28/-5','29/-5','32/-5','34/-6','38/-6','39/-6','42/-8',
             '13/-7','18/-3','22/-4','23/-9','25/-62','37/-5','40/-1','41/-6','20/-4',
             '26/-3','31/-6','33/-2','36/-6',
             '4/1','11/0','15/6','16/2','17/9','17/13','24/0','24/9','28/1','29/1','34/10','39/1','42/6',
             '18/13','22/0','23/12','25/1','37/1','41/14','26/12','31/6','33/7',
             '15/16','28/21','29/15','34/18','38/15','39/15','42/18','42/30',
             '25/15','37/17','37/22','40/17','26/27',
             '12/34','24/49','28/63','34/41',
             '18/91','21/50','22/91','23/61','25/31','31/58','33/47','36/38','36/63',
             '11/182','12/342','14/558','15/104','15/189','16/163','16/171','17/154',
             '19/153','24/127','28/237','29/186','39/115',
             '13/298','18.2/-9','18.2/16','18.2/71','21/120','23/130','37/138','26/185','31/178']


HEALTHY_SAMPLES = ['G1/-100','G2/-100','G3/-100','G4/-100','G5/-100','G6/-100',
             'G7/-100','G8/-100','G9/-100','G10/-100','G11/-100']

ZYMO_SAMPLES_NEW = ['Zymo_Zymo/-100','Zymo_EZ1/-100','Zymo_Power/-100','Zymo_Pro/-100']+['Stool_Power_11/-100',
 'Stool_Power_21/-100',
 'Stool_Pro_11/-100',
 'Stool_Pro_21/-100',
 'Stool_Zymo_11/-100',
 'Stool_Zymo_21/-100',
 'Stool_EZ1_12/-100',
 'Stool_EZ1_21/-100']

LIFELINES = ['LL81_E07_7627/-777', 'LL91_A04_8564/-777',
       'LL93_H07_8785/-777', 'LL87_A09_8215/-777', 'LL80_A10_7551/-777',
       'LL47_D03_4330/-777', 'LL63_B04_5872/-777', 'LL83_F03_7788/-777',
       'LL66_C11_6217/-777', 'LL80_D01_7482/-777', 'LL92_D01_8637/-777',
       'LL59_F11_5548/-777', 'LL83_G11_7853/-777', 'LL72_C02_6721/-777',
       'LL84_D03_7882/-777', 'LL45_F06_4164/-777', 'LL89_E11_8427/-777',
       'LL60_B04_5584/-777', 'LL45_F11_4204/-777', 'LL70_E06_6539/-777'
    
]


MARKER_SPECIES = {

    'Mucindegrading Bacteria':[
        'Akkermansia muciniphila',
        'Prevotella spp.'
    ],
    'Mucosaprotective':[
        'Akkermansia muciniphila',
        'Faecalibacterium prausnitzii'        
    ],

    'LPS associated':[
        'Citrobacter spp.',
        'Enterobacter spp.',
        'Escherichia spp.',
        'Klebsiella spp.',
        'Providencia spp.',
        'Pseudomonas spp.',
        'Serratia spp.',
        'Sutterella spp.'     
    ],

    'Immunomodulation':[
        'Escherichia spp.',
        'Enterococcus spp.'        
    ],

    'Clostridiaceae':[
        'Clostridium spp.'      
    ],

    'Fungi':[
        'Candida',
        'Geotrichum',
        'Saccharomyces cerevisiae'        
    ],
    
    'Blautia':[
        'Blautia'
    ]

}

MARKER_SPECIES_REDUCED = [
    'Faecalibacterium prausnitzii',
    'Akkermansia muciniphila',
    'Alistipes finegoldii',
    'Saccharomyces cerevisiae'
]

# Load Data <a class="anchor" id="loaddata"></a>

## Kraken2

In [ ]:
#This data can be collected from the snakemake workflow
kraken_dataframe = pd.read_csv('Input/KrakenFullDump.csv',usecols=[1,2,3,4,5,6],dtype={'patientid' : str,'taxonid':str})
adjust_table(kraken_dataframe)
kraken_dataframe = pd.concat([kraken_dataframe,
pd.read_csv('Input/KrakenLifelines.csv',usecols=[1,2,3,4,5,6],dtype={'patientid' : str,'taxonid':str}
            )]
                            
                            )
kraken_dataframe['sample'] = kraken_dataframe['patientid']+'/'+kraken_dataframe['time'].astype(str) 

## AMR Detection

In [ ]:
#can be extracted from the snakemake workflow, used to normalize the AMR read counts
sequencing_stats = pd.read_csv('Input/sampleStats.filtered.csv',dtype={'patientid' : str})

sequencing_stats['sample'] = sequencing_stats['patientid']+'/'+sequencing_stats['time'].astype(str)

adjust_table(sequencing_stats)
sequencing_stats = sequencing_stats[sequencing_stats['sample'].isin(PAPER_SAMPLES)]

readcount_min = sequencing_stats['nReads'].min()

sequencing_stats = sequencing_stats.set_index(['sample'])

def calc_fraction(x):
    if x['sample'] in sequencing_stats.index:
        return x['readcount'] / sequencing_stats.loc[x['sample']]['nReads']
    else:
        return np.NaN

#can be extracted from the snakemake workflow
amr = pd.read_csv('Input/fulldump_amr_170322.csv',dtype={'Patient' : str}).rename(
columns={'Patient' : 'patientid','Time' : 'time'})

amr['sample'] = amr['patientid'].astype(str)+'/'+amr['time'].astype(str)
amr['readcount'] = amr.apply(
    lambda x : 1/x['Ambiguous Assignments'] if x['Ambiguous Assignments'] != 0 else 1,axis=1
)
adjust_table(amr)
amr = amr[amr['sample'].isin(PAPER_SAMPLES)]

amr['Gene Pro Read'] =amr.apply(lambda x : calc_fraction(x) ,axis=1)



## Kraken2/Metamaps Taxonomy

In [ ]:
namesToIDs = {
    'unclassified' : -1,
    'Unassigned at Level' : '-1337'
}

with open('Input/Taxonomy/names.dmp','r') as f:
    for l in f.read().splitlines():
        d=[x.strip() for x in l.split('|')]
        if d[3] == 'scientific name':
            namesToIDs[d[1]] = d[0]
with open('Input/Taxonomy/names_additional.dmp','r') as f:
    for l in f.read().splitlines():
        d=[x.strip() for x in l.split('|')]
        if d[3] == 'scientific name':
            namesToIDs[d[1]] = d[0]

taxonomy = {}


levels = {}

with open('Input/Taxonomy/nodes.dmp','r') as f:
    for l in f.read().splitlines():
        d=[x.strip() for x in l.split('|')]
        taxonomy[d[0]] = d[1]
        levels[d[0]] = d[2]
        
with open('Input/Taxonomy/nodes_additional.dmp','r') as f:
    for l in f.read().splitlines():
        d=[x.strip() for x in l.split('|')]
        taxonomy[d[0]] = d[1]        
        levels[d[0]] = d[2]

# ID to names
idtonames = {
        v : k 
        for k, v in namesToIDs.items()
}

idtonames['46506']  = 'Bacteroides stercoris'
idtonames['820']  = 'Bacteroides uniformis'
idtonames['4335590'] = 'Bacteroides Vulgatus'
idtonames['1118061'] = 'Alistipes communis'

## Zymo Theory

In [ ]:
zymo_theory = pd.read_csv('Input/zymo_theory.csv',
                          header=None,
                          names=['Taxon','Read Fraction']
                         )

zymo_theory_taxa = zymo_theory['Taxon'].tolist()
zymo_theory['Read Fraction (%)'] = zymo_theory['Read Fraction']/100
zymo_theory['Sample ID'] = 'Zymo Theoretical Composition'

zymo_minimap = pd.read_csv('Input/zymo_minimap_verification.csv')

## Metamaps

In [ ]:
metamaps_dataframe = pd.read_csv(
    'Input/fulldumpMetamaps_2021-08-19.csv',
    usecols=[1,2,3,4,5],
    dtype={'patientid' : str}
) 

## Load Sample Annotations

In [ ]:
#PatID is specified as String (Text) so IDs like 18.2 don't get confused as decimal numbers

#Both tables contain annotations regarding the individual samples
sample_characteristics = pd.read_excel('Input/Annotations/Probencharakteristiken_030122.xlsx',dtype={'PatID' : str})
sample_statistics = pd.read_excel('Input/Annotations/Probenstatistiken_010322.xlsx',dtype={'PatID' : str})

#Subset of clinical data containing the leukocyte values
leukocytes = pd.read_csv('Input/Annotations/KI_Patienten_Nur_Leukos.csv',sep=';')
leukocytes.loc[
    ((leukocytes['KI ID']==18)&(leukocytes['relatives_datum']>92)),'KI ID'
] = '18.2'
leukocytes.loc[(leukocytes['KI ID']=='18.2'),'relatives_datum'] -= OFFSET_PAT_18

#Outcomes
outcomes = pd.read_excel('Input/Annotations/Patient_Statistics (2).xlsx',dtype={'Pat ID' : str})

# Normalization Function <a class="anchor" id="normalize"></a>

In [ ]:
def get_normalized_abundances(
    kraken_dataframe,
    level, #No default value here to avoid accidental mistakes!
    samples = None,
    random_seed = (4+8+15+16+23+42),
    normalize=True,
    excluded_taxa_filter = None, #Can be a list of taxa
    included_taxa_filter = None, #Only one taxa, becomes new root
):
    
    ### Helper Functions for Filtering
    
    not_found_taxa = set()

    @lru_cache(maxsize=None)
    def is_not_below(x,y):
        parent_node = taxonomy[x]
        if parent_node == y: #Parent was the node we looked for
            return False
        elif parent_node == x: #This is only the case at the root node
            return True
        else: #We need to keep looking
            return is_not_below(parent_node,y)


    def filter_function(row,taxon):
        if row['taxonid'] == '0': #Root Node
            return False
        if row['taxonid'] not in taxonomy:
            #print('Warning: TaxID {} is not in the taxonomy, this is (potentially) bad!'.format(row['taxonid']))
            return False
        if row['taxonid'] == taxon: #This is the taxon itself, remove
            return False
        #Otherwise we will check if the taxon is below our target taxon in the taxonomy
        return is_not_below(row['taxonid'],taxon)


    ###
    
    #We begin by assuming the raw kraken dataframe as input
    working_table = kraken_dataframe
    
    #####################
    #   SELECT SAMPLES
    #
    #####################    
    
    #Filter to target samples
    if samples != None:
        working_table = working_table[working_table['sample'].isin(samples)]    
    

    #####################
    #   CALCULATE UNASSIGNED READS
    #
    #####################    
    
    #Determine root readcounts
    root_readcounts_kr = None
    
    if included_taxa_filter != None:
        root_readcounts_kr = working_table[
            working_table['taxonid']==included_taxa_filter
        ].groupby(
            ['sample']
        )['readcount'].sum()    
    else: #If we don't filter we take all reads -> R
        root_readcounts_kr = working_table[
            working_table['level']=='R'
        ].groupby(
            ['sample']
        )['readcount'].sum()
        
    readcounts_at_level=None
    
    if included_taxa_filter != None:
        include = working_table[
            working_table['level'] == level
        ].apply(lambda x : filter_function(x,included_taxa_filter),axis=1)
        
        readcounts_at_level = working_table[
            working_table['level'] == level
        ][~include]
        
        readcounts_at_level = readcounts_at_level.groupby(
            ['sample']
        )['readcount'].sum()
    else:
        #Determine total read counts at level
        readcounts_at_level = working_table[
            working_table['level'] == level
        ].groupby(
            ['sample']
        )['readcount'].sum()
    
    #Add unassigned at level
    unassigned_entries = []
    for sample in readcounts_at_level.keys():
        
        patientid,time = sample.split('/')
        time = int(time)

        difference = root_readcounts_kr[sample]-readcounts_at_level[sample]
        
        addon_table = pd.DataFrame([
            (difference,patientid,time,'Unassigned at Level','-1337',level,sample)
        ],columns=[
            'readcount','patientid','time','taxon','taxonid','level','sample'
        ])
        
        unassigned_entries.append(addon_table)
                
    #Combine into one table
    if len(unassigned_entries) != 0:
        unassigned_table = pd.concat(unassigned_entries)
        working_table = pd.concat([unassigned_table,working_table])
    
    #####################
    #   DOWNSAMPLING
    #
    #####################
    
    #Filter to target level
    downsampled_table = working_table[working_table['level'] == level]  
    
    #Filter for taxon if required
    if included_taxa_filter != None:
        print('Reducing composition to subtree below taxon: {}'.format(idtonames[included_taxa_filter]))
        include = downsampled_table.apply(lambda x : filter_function(x,included_taxa_filter),axis=1)
        downsampled_table = downsampled_table[~include]
           
    if normalize:
    
        # Identify lowest read count
        readAnzahlen = downsampled_table.groupby('sample')['readcount'].sum()
        minimaleReadAnzahl = readAnzahlen.min()
        print('The minimal read count across all samples is [Taxonomic Level {}]: {}'.format(level,minimaleReadAnzahl))

        frames = []

        # Draw new counts for each sample
        for probe in downsampled_table['sample'].unique():

            sample_table = downsampled_table[downsampled_table['sample'] == probe]
            sample = sample_table.sample(
                n=round(minimaleReadAnzahl),
                random_state=random_seed,
                weights='readcount',
                replace=True
            )

            sample = sample.groupby([
                'taxon',
                'taxonid',
                'sample',
                'patientid',
                'time',
                'level'
            ],as_index=False).count()

            frames.append(sample)

        #Overwrite table with downsampled entries
        downsampled_table = pd.concat(frames)   
    
    #####################
    #   FILTERING II
    #
    #####################
    
    unassigned_downsampled_table = downsampled_table[downsampled_table['taxonid'] == '-1337']
    assigned_downsampled_table = downsampled_table[downsampled_table['taxonid'] != '-1337']
    
    

    if excluded_taxa_filter != None:
        for taxon in excluded_taxa_filter:
            include = assigned_downsampled_table.apply(lambda x : filter_function(x,taxon),axis=1)
            assigned_downsampled_table = assigned_downsampled_table[include]

        
    downsampled_table = pd.concat([unassigned_downsampled_table,assigned_downsampled_table])
        
        

    return downsampled_table


# Top Level Statistics Visualization <a class="anchor" id="toplevel"></a>

In [ ]:
Gruppen=['Bacteria', 'Fungi','Other eukaryota','Viruses','Archaea']
z = sample_statistics[['leukocytephase_cluster_2_kurz','Bacteria_Prozent pro classified ohne human','Fungi_Prozent pro classified ohne human','Other eukaryota_Prozent pro classified ohne human','Viruses_Prozent pro classified ohne human','Archaea_Prozent pro classified ohne human','Probe']]
z = z.melt(id_vars=['Probe','leukocytephase_cluster_2_kurz'])
z[['domain','variable']] = z['variable'].str.split('_',expand=True)
z = z.drop(columns=['variable'])
z = z.pivot(index=['Probe','leukocytephase_cluster_2_kurz'],columns='domain',values='value')
z = z.reset_index()
z = z.melt(id_vars=['Probe','leukocytephase_cluster_2_kurz'])
z['domain'].unique()
Tabelle_pre=sample_statistics[sample_statistics['leukocytephase_cluster_2_kurz'].isin(['Healthy', 'Pre TX','Leukozytopenia','Reconstitution'])]
c1 = alt.Chart(sample_statistics,width=100,height=100).mark_boxplot(ticks=True,median={'color':'black'}).encode(
    column=alt.Column(
        'leukocytephase_cluster_2_kurz:O',
        header=alt.Header(labels=False),
        spacing=0,
        sort=['Healthy', 'Pre TX','Leukozytopenia','Reconstitution'],
        title=None
    ),
    y=alt.Y('Mikrogramm DNA pro g Stuhl ', axis=alt.Axis(grid=False,minExtent=40), title='DNA [µg] Per g Stool'),

    )



c2 = alt.Chart(sample_statistics,width=100,height=100).mark_boxplot(ticks=True,median={'color':'black'}).encode(
    column=alt.Column(
        'leukocytephase_cluster_2_kurz:O',
        header=alt.Header(labels=False),
        spacing=0,
        sort=['Healthy', 'Pre TX','Leukozytopenia','Reconstitution'],
        title=None
    ),
     y=alt.Y('Total reads pro DNA Library:Q',scale=alt.Scale(type='log'), axis=alt.Axis(grid=False,tickCount=10, format='.1e',minExtent=40),title='Total Reads')
    )

c2_2 = alt.Chart(sample_statistics,width=100,height=100).mark_boxplot(ticks=True,median={'color':'black'}).encode(
    column=alt.Column(
        'leukocytephase_cluster_2_kurz:O',
        header=alt.Header(labels=False),
        spacing=0,
        sort=['Healthy', 'Pre TX','Leukozytopenia','Reconstitution'],
        title=None
    ),
     y=alt.Y('median length alle reads:Q',scale=alt.Scale(type='log'), axis=alt.Axis(grid=False,tickCount=10, format='.1e',minExtent=40),title='Median Readlength')
    )


c3 = alt.Chart(sample_statistics,width=100,height=100).mark_boxplot(ticks=True,median={'color':'black'}).encode(
    column=alt.Column(
        'leukocytephase_cluster_2_kurz:O',
        header=alt.Header(labels=False),
        spacing=0,
        sort=['Healthy', 'Pre TX','Leukozytopenia','Reconstitution'],
        title=None
    ),
    y=alt.Y('Unclassified_Prozent pro total reads', axis=alt.Axis(grid=False,minExtent=40), title='Unclassified [% Reads]')
    ) 

c4 = alt.Chart(sample_statistics,width=100,height=100).mark_boxplot(ticks=True,median={'color':'black'}).encode(
    column=alt.Column(
        'leukocytephase_cluster_2_kurz:O',
        header=alt.Header(labels=False),
        spacing=0,
        sort=['Healthy', 'Pre TX','Leukozytopenia','Reconstitution'],
        title=None
    ),
    y=alt.Y('Human_Prozent pro total reads', axis=alt.Axis(grid=False,minExtent=40), title='Human [% Reads]')
    )

c5 = alt.Chart(sample_statistics,width=100,height=100).mark_boxplot(ticks=True,median={'color':'black'}).encode(
    column=alt.Column(
        'leukocytephase_cluster_2_kurz:O',
        header=alt.Header(labels=False),
        spacing=0,
        sort=['Healthy', 'Pre TX','Leukozytopenia','Reconstitution'],
        title=None
    ),
    y=alt.Y('G_False', axis=alt.Axis(grid=False,minExtent=40), title='No. Genera')
    )

c6 =alt.Chart(Tabelle_pre,width=100,height=100).mark_boxplot(ticks=True,median={'color':'black'}).encode(
    column=alt.Column(
        'leukocytephase_cluster_2_kurz:O',
        spacing=0,
        sort=['Healthy', 'Pre TX','Leukozytopenia','Reconstitution'],
        title=None,
        header=alt.Header(labels=False)
    ),
    y=alt.Y('ARG Reads pro Eingabe Reads *10000:Q', title=['ARG-carrying Reads','Per 10,000 Reads'],scale=alt.Scale(type='symlog'), axis=alt.Axis(grid=False,minExtent=40))
    )



c7 = alt.Chart(z,width=100).mark_boxplot(ticks=True,median={'color':'black'}).encode(
    column=alt.Column(
        'leukocytephase_cluster_2_kurz:O',
        spacing=0,
        sort=['Healthy', 'Pre TX','Leukozytopenia','Reconstitution'],
        title=None,
        header=alt.Header(labelOrient='bottom', labelPadding=313)
    ),
     x=alt.X("domain:O", title=None, axis=alt.Axis(labels=False, ticks=False), scale=alt.Scale(paddingInner=1), sort=Gruppen),    
    y=alt.Y("value:Q",title='Fraction [%]',scale=alt.Scale(type='symlog',domain=[0,100]), axis=alt.Axis(grid=False,minExtent=40)), 
    color=alt.Color("domain:N",sort=Gruppen,legend=alt.Legend(title=None,orient='top'),
                    scale=alt.Scale(domain=Gruppen,range=['#EE6677', '#228833', '#CCBB44', '#66CCEE', '#AA3377'])),

)

chart1=(c1&c2&c2_2&c3&c4&c5&c6&c7).configure_tick(thickness=2)

chart1.save(
    'Output/Composition/High_Level_Metrics.html')

chart1

# Top Level Statistics P-Values <a class="anchor" id="pvalues"></a>

In [ ]:
test_column = 'Chordata_Prozent pro classified'
group_column = 'Startcluster'

tuples_a = []
for group1,group2 in it.combinations(sample_statistics[group_column].unique(),2):
    result = mannwhitneyu(
        sample_statistics[sample_statistics[group_column] == group1][test_column],
        sample_statistics[sample_statistics[group_column] == group2][test_column]  
    )
    tuples_a.append((group1,group2,result.pvalue))
    
df_a = pd.DataFrame(tuples_a,columns=['Group A','Group B','Whitney'])
df_a.to_csv('Output/Whitney_{}_{}.csv'.format(test_column,group_column))
df_a

tuples_b = []
for group in sample_statistics[group_column].unique():
    tuples_b.append((
        group,
        sample_statistics[sample_statistics[group_column] == group][test_column].mean(),
        sample_statistics[sample_statistics[group_column] == group][test_column].std(),
        sample_statistics[sample_statistics[group_column] == group][test_column].min(),
        sample_statistics[sample_statistics[group_column] == group][test_column].max()
        
    ))
df_b = pd.DataFrame(tuples_b,columns=['Group','Mean','Standardabweichung','Minimum','Maximum'])
df_b.to_csv('Output/MeanAndStd_{}_{}.csv'.format(test_column,group_column))
df_b

# Overview Top Taxa per Taxonomic Level <a class="anchor" id="toptaxa"></a>

In [ ]:
TOP_X = 20
LEVEL = 'S'
SAMPLES = PAPER_SAMPLES
NORMALIZE = True
EXCLUDE = None
INCLUDE = None

#################


hash_samples = hash(str(SAMPLES))

df_filtered = get_normalized_abundances(kraken_dataframe,level=LEVEL,included_taxa_filter=INCLUDE,excluded_taxa_filter=EXCLUDE,samples=SAMPLES,normalize=NORMALIZE)
df_filtered['Read Fraction'] = df_filtered['readcount'] / df_filtered.groupby(['time','patientid'])['readcount'].transform('sum')

taxa_we_look_at = list(df_filtered.groupby('taxon')['Read Fraction'].mean().sort_values(ascending=False)[:(TOP_X+1)].keys())
df_filtered = df_filtered[df_filtered['taxon'].isin(taxa_we_look_at)]

df_filtered = df_filtered.pivot(index=['patientid','time'],values=['Read Fraction'],columns=['taxon']).fillna(0)

df_filtered.to_csv('Output/Top_{}_Taxa_{}_{}_{}-Without_{}.csv'.format(TOP_X,LEVEL,hash_samples,'Normalized' if NORMALIZE else 'Raw',str(EXCLUDE)))

df_filtered


# Unmapped and Human Fractions <a class="anchor" id="human"></a>

In [ ]:
unmapped = kraken_dataframe[kraken_dataframe['level'] == 'U'].set_index(['patientid','time'])['readcount'].rename('Unmapped')
mapped = kraken_dataframe[kraken_dataframe['level'] == 'R'].set_index(['patientid','time'])['readcount'].rename('Mapped')

chordata = get_normalized_abundances(
    kraken_dataframe,
    level='P',
    included_taxa_filter=CHORDATA,
    normalize=False,
    samples=None
)

chordata = chordata[chordata['taxon'] == 'Chordata'].set_index(['patientid','time'])['readcount'].rename('Chordata')

combined = pd.concat([unmapped,mapped,chordata],axis=1).fillna(0)
combined['Other Classified'] = combined['Mapped']-combined['Chordata']

combined = combined[['Unmapped','Chordata','Other Classified']]
combined = combined.div(combined.sum(axis=1), axis=0)

combined = combined.reset_index()

combined.to_csv('Output/Unmapped_Mapped_Chordata_Other.csv')

combined

In [ ]:
combined['sample'] = combined['patientid']+'/'+combined['time'].astype(str)

data = combined[['sample','Unmapped','Chordata','Other Classified']].melt(id_vars=['sample'])

alt.Chart(data).mark_bar().encode(
    x='sample',
    y='value',
    color='variable',
    tooltip=['sample','value','variable']
).save('Output/Unmapped_Chordata_Other.html')

# AMR Gene Counts Calculations <a class="anchor" id="amr"></a>

In [ ]:
gene_per_read=amr.groupby(['patientid','time'])['readcount'].sum()

gene_per_read.to_excel('Output/gene_per_read.xlsx',index=False)

gene_per_read

# Taxa Counts (Diversity) Per Level <a class="anchor" id="taxacount"></a>

In [ ]:
SAMPLES = PAPER_SAMPLES
EXCLUDE = [CHORDATA]
############################

os.makedirs('Output/Diversity',exist_ok=True)

charts = []

for taxonomic_level in ['S','G','P']:
    table = get_normalized_abundances(
        kraken_dataframe,
        level=taxonomic_level,
        excluded_taxa_filter=EXCLUDE,
        samples=SAMPLES
    )
    
    table = table[table['taxonid']!='-1337']
     
    taxaCounts = table.groupby(['patientid','time'],as_index=False)['taxon'].count().rename(
        columns={'taxon' : '{}_{}'.format(taxonomic_level,'Normalized')}
    )
    
    charts.append(taxaCounts)


reduce(lambda x,y : pd.merge(x,y,how='left',on=['patientid','time']),charts).to_csv(
    'Output/Diversity/taxa_counts_{}-Without_{}_Normalized.csv'.format(
        hash(str(SAMPLES)),
        str(EXCLUDE)
    )
)


charts = []


for taxonomic_level in ['S','G','P']:
    table = get_normalized_abundances(
        kraken_dataframe,
        level=taxonomic_level,excluded_taxa_filter=EXCLUDE,normalize=False)
    
    table = table[table['taxonid']!='-1337']
    
    taxaCounts = table.groupby(['patientid','time'],as_index=False)['taxon'].count().rename(
        columns={'taxon' : '{}_{}'.format(taxonomic_level,'Raw')}
    )
    
    charts.append(taxaCounts)
        
reduce(lambda x,y : pd.merge(x,y,how='left',on=['patientid','time']),charts).to_csv(
    'Output/Diversity/taxa_counts_{}-Without_{}_RAW.csv'.format(
        hash(str(SAMPLES)),
        str(EXCLUDE)
    )
)


# Barplots Compositions <a class="anchor" id="barplots"></a>

In [ ]:
TOP_X = 20
LEVEL = 'S'
#Filters
INCLUDE = None
EXCLUDE = [CHORDATA]

SAMPLES = PAPER_SAMPLES

RELATIVE = True # If False, abundances are relative to root

OPTIONAL_ANNOTATIONS = None

SORTING = 'Sample ID' #Default='sample'

GROUPING = None#'Startcluster' # None if you don't want any Grouping, otherwise a column to group by

NORMALIZE = True

DISCARD_CUTOFF = 100
#################

os.makedirs('Output/Composition',exist_ok=True)

input_table = get_normalized_abundances(
    kraken_dataframe,
    level=LEVEL,
    samples=SAMPLES,
    included_taxa_filter=INCLUDE,
    excluded_taxa_filter=EXCLUDE,
    normalize=NORMALIZE
)


if RELATIVE:
    input_table['Read Fraction (%)'] = input_table['readcount'] / input_table.groupby(['time','patientid'])['readcount'].transform('sum')
else:
    root_levels = kraken_dataframe[kraken_dataframe['level'] == 'R']
    root_levels = root_levels[root_levels['sample'].isin(input_table['sample'].unique())][['sample','readcount']].rename(columns={'readcount':'Reads At Root'})
    input_table = pd.merge(input_table,root_levels,how='left',on='sample')
    input_table['Read Fraction (%)'] = input_table['readcount'] / input_table['Reads At Root']
    input_table = input_table.drop(columns=['Reads At Root'])
    
# determine the top taxa based on means
taxa_we_look_at = list(input_table.groupby('taxon')['Read Fraction (%)'].mean().sort_values(ascending=False)[:(TOP_X+1)].keys())+['Unassigned at Level']
print('Determined the following taxa as relevant:',taxa_we_look_at)

total_reads = input_table.groupby('sample',as_index=False)['readcount'].sum()
masked_samples = total_reads[total_reads['readcount'] < DISCARD_CUTOFF]['sample']


#assign everything else to the "other" group and readjust sum
input_table.loc[~input_table['taxon'].isin(taxa_we_look_at), 'taxon'] = 'Other'
input_table.loc[input_table['sample'].isin(masked_samples), 'taxon'] = 'Not enough reads'

input_table = input_table.groupby(['taxon','time','patientid'],as_index=False).sum()

input_table['id'] = (
    input_table['patientid'].astype(str)+'/'+input_table['time'].astype(str)
)

input_table['other'] = input_table['taxon'] == 'Other'

input_table = input_table.rename(columns={
    'id' : 'Sample ID',
    'taxon' : 'Taxon'
})

colorMap = {}

taxa = input_table['Taxon'].unique()

palette = sns.color_palette("tab20",n_colors=len(taxa)) #hls

for tax,col in zip(list(taxa),palette):
    colorMap[tax] = col #colors.to_hex(col)
    
altdomain = []
altrange = []

for x in input_table['Taxon'].unique():
    
    if x == 'Other' or x == 'Unassigned at Level' or x == 'Not enough reads':
        continue

    c = colorMap[x]
    altdomain.append(x)
    color = colors.to_hex(c)
    altrange.append(color)
    
altdomain.append('Other')
altrange.append(colors.to_hex((1,1,1)))
altdomain.append('Unassigned at Level')
altrange.append(colors.to_hex((0.15,0.15,0.15)))
altdomain.append('Not enough reads')
altrange.append(colors.to_hex((0.55,0.55,0.55)))

input_table = pd.merge(input_table,sample_statistics,how='left',left_on=['patientid','time'],right_on=['PatID','time'])

input_table['Aktuelles Cluster'] = input_table['Aktuelles Cluster'].fillna('Unassigned')

charts = []

groups = input_table[GROUPING].unique() if GROUPING != None else [None]

for group in groups:
    
    grouptable = None
    if group != None:
        #Reduce to required columns to keep output reasonably small
        grouptable = input_table[input_table['Aktuelles Cluster'] == group]
    else:
        grouptable = input_table
    
    patientlist_sorted = sorted(
        grouptable[SORTING].unique().tolist()
    )
    
    chart =  alt.Chart(
          grouptable,title=str(group)
      ).transform_calculate(
      order=f"-indexof({altdomain}, datum.Taxon)"
        ).mark_bar(stroke='black',strokeWidth=0.5,strokeOpacity=0.9).encode(
            x=alt.X('{}:N'.format(SORTING),sort=patientlist_sorted),
            y=alt.Y('Read Fraction (%):Q',scale=alt.Scale(
                domain=(0,1))
                   ),
            color=alt.Color('Taxon:N',legend=alt.Legend(columns=1,symbolLimit=0,labelLimit=0),scale=alt.Scale(domain=altdomain,range=altrange)),
            tooltip=['readcount','Read Fraction (%)','Taxon'],
            order=alt.Order('order:Q')
      )
    
    if OPTIONAL_ANNOTATIONS != None:
        for optional_annotation in OPTIONAL_ANNOTATIONS:
            chart &= alt.Chart(grouptable,title=str(group)).mark_bar(stroke='black',strokeWidth=0.5,strokeOpacity=0.9).encode(
                x=alt.X('{}:N'.format(SORTING),sort=patientlist_sorted),
                y=alt.Y('{}'.format(optional_annotation))
            )

    charts.append(
        chart
         
        
    )

chart = reduce(lambda x,y : x&y, charts).configure_axis(
    labelFontSize=16, titleFontSize=16
).configure_title(fontSize=20).configure_legend(titleFontSize=20, labelFontSize=16)

chart.save(
    'Output/Composition/Barplots-Top_{}-{}-Only_{}-Without_{}-{}-{}_{}_{}_{}.html'.format(
        TOP_X,
        LEVEL,
        INCLUDE,
        str(EXCLUDE),
        hash(str(SAMPLES)),
        'Relative' if RELATIVE else 'Absolute',
        hash(str(OPTIONAL_ANNOTATIONS)),
        SORTING,
        GROUPING
    )
)

chart

In [ ]:
input_table_pivot = input_table.pivot(index=['time','patientid'],columns='Taxon',values='Read Fraction (%)')

input_table_pivot.transpose()

input_table_pivot=input_table_pivot.fillna(0)

input_table_pivot.to_csv(
    'Output/Composition/Compositions-Top_{}-{}-Only_{}-Without_{}-{}-{}_{}_{}_{}.csv'.format(
        TOP_X,
        LEVEL,
        INCLUDE,
        str(EXCLUDE),
        hash(str(SAMPLES)),
        'Relative' if RELATIVE else 'Absolute',
        hash(str(OPTIONAL_ANNOTATIONS)),
        SORTING,
        GROUPING
    )
)

# Patient Journeys <a class="anchor" id="barplots_time"></a>

In [ ]:
TOP_X = 20

os.makedirs('Output/Patient_Journey',exist_ok=True)

### Prepare Annotations

sample_statistics['leukocytephase_cluster_2_kurz_kuerzer']=sample_statistics[
    'leukocytephase_cluster_2_kurz'
].replace('Leukozytopenia','N').replace('Pre TX','P').replace('Reconstitution','R')

outcomes_reduced = outcomes[
    ['Pat ID',
     'Day relative to HSCT',
     'Day relative to HSCT.1',
     'Day relative to HSCT.2',
     'Day relative to HSCT.3']
         ].rename(
    columns={
        'Day relative to HSCT' : 'Relapse',
        'Day relative to HSCT.1' : 'Acute GvHD Grade 1-2',
        'Day relative to HSCT.2' : 'Acute GvHD Grade 3-4',
        'Day relative to HSCT.3' : 'Death',
    }
)
outcomes_reduced['No Adverse Event']=outcomes_reduced.apply( lambda x : 0 if x.count() <= 1 else np.NaN,axis=1)
outcomes_reduced

outcomes_reduced = outcomes_reduced.melt(id_vars=['Pat ID'])
outcomes_reduced = outcomes_reduced[(outcomes_reduced['value']==outcomes_reduced['value'])]

outcomes_reduced.loc[
    (outcomes_reduced['Pat ID']=='18.2'),'KI ID'
] = '18.2'
outcomes_reduced.loc[(outcomes_reduced['Pat ID']=='18.2'),'value'] -= OFFSET_PAT_18

#Load Marker Gene Data

print('Load Marker Gene Data')

specframe = kraken_dataframe[kraken_dataframe['level'] == 'S']

specframe=specframe[specframe['sample'].isin(PAPER_SAMPLES)] 
specframe['estimated abundance'] = (specframe['readcount'] / specframe.groupby(['time','patientid'])['readcount'].transform('sum')).round(10)

marker_species = {}

for group in MARKER_SPECIES:
    marker_species[group] = specframe[
        specframe['taxon'].str.startswith(tuple(MARKER_SPECIES[group]))
    ]
    marker_species[group]['time2'] = marker_species[group]['time'] + 1

charts = []

#### Shannon Block
shannon_df = get_normalized_abundances(kraken_dataframe,samples=PAPER_SAMPLES,level=LEVEL,excluded_taxa_filter=[CHORDATA],normalize=True)
def shannon(values):
    max_shannon = math.log(len(values))
    shannon = -sum(x*math.log(x) for x in values)
    shannon_quotient = shannon/max_shannon
    cols = ['Shannon Index','Shannon Max Value','Shannon Quotient']
    return pd.Series((shannon,max_shannon,shannon_quotient),index=cols)

shannon_df['Read Fraction'] = shannon_df['readcount']/shannon_df.groupby(['patientid','time'])['readcount'].transform('sum')
shannon_df = shannon_df.groupby('sample',as_index=False)['Read Fraction'].apply(func= lambda x : shannon(x))
shannon_df['time'] = shannon_df['sample'].str.split('/',expand=True)[1]

for idx,row in sample_statistics.iterrows():
    
        
    patient = row['PatID']
    samples = [x for x in PAPER_SAMPLES if x.split('/')[0] == str(patient)]
   
    cluster = row['leukocytephase_cluster_kurz'].split('_')[1]
    
    compcharts = []
    
    for taxonomic_level in ['P','G','S']:
        
        for filter_option in [CHORDATA,FUNGI,VIRUSES]:
            if filter_option == CHORDATA:
                input_table = get_normalized_abundances(kraken_dataframe,level=taxonomic_level,excluded_taxa_filter=[CHORDATA],samples=samples)
            else:
                input_table = get_normalized_abundances(kraken_dataframe,level=taxonomic_level,included_taxa_filter=filter_option,samples=samples)

            #input_table = input_table[input_table['time'].between(ZEIT_CUTOFF_1,ZEIT_CUTOFF_2)]

            input_table['readAnteil'] = input_table['readcount'] / input_table.groupby(['time','patientid'])['readcount'].transform('sum')


            input_table = pd.merge(input_table,sample_statistics,how='left',left_on=['patientid','time'],right_on=['PatID','time'])

            input_table = pd.merge(input_table,sample_characteristics,how='left',left_on=['patientid','time'],right_on=['PatID','time'])

            input_table['Startcluster'] = input_table['leukocytephase_cluster_kurz_x'].apply(lambda x: x.split('_')[1])

            input_table = input_table.rename(columns={
                'readAnteil' : 'Read Fraction (%)',
                'taxon' : 'Taxon'
            })

            input_table['Leukozyten*1000proÂµL_Abnahme']=input_table['Leukozyten*1000proÂµL_Abnahme'].apply(lambda x : 0 if isinstance(x,str) else x)
            input_table['readlength kbp']=input_table['median length alle reads']/1000

            input_table['timeplus1'] = input_table['time']+1


            le_tableaux = input_table
            le_tableaux = le_tableaux[le_tableaux['patientid'] == patient]
            taxa_we_look_at = le_tableaux.groupby('Taxon')[
                'Read Fraction (%)'
            ].mean().sort_values(ascending=False)[:(TOP_X+1)].keys()
            taxa_we_look_at = list(taxa_we_look_at) + ['Unassigned at Level']
            #print(taxa_we_look_at)
            le_tableaux.loc[~le_tableaux['Taxon'].isin(taxa_we_look_at), 'Taxon'] = 'Other'
            le_tableaux = le_tableaux.groupby(['Taxon','time','patientid'],as_index=False).sum()
        
            #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
            colorMap = {}

            taxa = le_tableaux['Taxon'].unique()

            palette = sns.color_palette("tab20",n_colors=len(taxa)) #hls

            for tax,col in zip(list(taxa),palette):
                colorMap[tax] = col #colors.to_hex(col)

            altdomain = []
            altrange = []

            for x in le_tableaux['Taxon'].unique():

                if x == 'Other' or x == 'Unassigned at Level':
                    continue

                c = colorMap[x]
                altdomain.append(x)
                color = colors.to_hex(c)
                altrange.append(color)

            altdomain.append('Other')
            altrange.append(colors.to_hex((1,1,1)))
            altdomain.append('Unassigned at Level')
            altrange.append(colors.to_hex((0,0,0)))
            #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

            compchart = alt.Chart(le_tableaux,width=700,height=130,title=[
                 '{}'.format(
                {'P':'Phylum','S':'Species','G':'Genus'}[taxonomic_level]),{
                    FUNGI : 'Only Fungi',
                    VIRUSES : 'Only Viruses',
                    CHORDATA : 'Total (Filtered for Human)'
                }[filter_option]]
                    ).transform_calculate(
                      order=f"-indexof({altdomain}, datum.Taxon)"
                    ).mark_bar(stroke='black',strokeWidth=0.5,strokeOpacity=0.9).encode(
                            x=alt.X('time:Q',title=None,axis=alt.Axis(tickMinStep=1)),
                            y=alt.Y('Read Fraction (%):Q',title=['Read Fraction (%)',''],
                                    scale=alt.Scale(domain=(0,1)),stack=True),
                            order = 'order:Q',
                            color=alt.Color('Taxon:N', legend=alt.Legend(columns=4,
            orient='top',
            legendX=0, legendY=-140,
            direction='horizontal',
            titleAnchor='middle'),scale=alt.Scale(domain=altdomain,range=altrange)),
                            tooltip=['readcount','Read Fraction (%)','patientid','Taxon','time'],
                        )
            compcharts.append(compchart)
        
    patientsub = sample_statistics[sample_statistics['PatID']== patient]

    phasechart = (alt.Chart(sample_statistics[sample_statistics['PatID']== patient],width=700,title='Leukocyte Phase (Pre-TX / Nadir / Reconstitution)').mark_text(fontStyle='bold').encode(
                    x=alt.X('time:Q',axis=None),
                    text='leukocytephase_cluster_2_kurz_kuerzer:N'
                ))
    
    mscharts = []
    for group in marker_species:
        ms_group_table = marker_species[group]
        ms_group_table['primary identifier'] = ms_group_table['taxon'].apply(lambda x: x.split(' ')[0])
        ms_group_table = ms_group_table.groupby(['primary identifier','patientid','time','time2'],as_index=False)['estimated abundance','readcount'].sum()

        c =alt.Chart(ms_group_table[ms_group_table['patientid'] == patient],title='Marker Species: '+group).mark_point(shape='square',filled=True).encode(
            x=alt.X('time:Q',title=None),
            x2=alt.X2('time2:Q'),
            y=alt.Y('primary identifier'),
            color=alt.Color('estimated abundance',title='Estimated Abundance (Not Normalized)'),
            tooltip=['estimated abundance','readcount']
        )
        mscharts.append(c)
    
    patientoutcomes= outcomes_reduced[outcomes_reduced['Pat ID'] == patient]
    outcome_set = list(set(patientoutcomes['value'])-{'?'})
    last_outcome=max(outcome_set) if len(outcome_set)>0 else -999
    last_leuko=max(patientsub['time'].max()+5,last_outcome)


    leukosub = leukocytes[leukocytes['relatives_datum'].between(patientsub['time'].min()-5,last_leuko)]
    leukosub = leukosub[leukosub['KI ID'].astype(str) == patient]
    leukochart= alt.Chart(leukosub,height=130,width=700).mark_line(point=True).encode(
        x=alt.X('relatives_datum:Q',axis=alt.Axis(grid=False,tickMinStep=1),title='Day'),
        y=alt.Y('Messwert_Zahl',title=['Leukocytes','(1000/µL)']),
        order='relatives_datum'
    )+alt.Chart(patientsub).mark_rule().encode(
        x='time'
    )
    
    #### OUTCOMES

    outcomechart = None
    if 'No Adverse Event' in patientoutcomes['variable'].unique():
        outcomechart = alt.Chart({'values':[{}]},width=700).mark_text(
        align="left", baseline="top",fontStyle='bold',
                fontSize=12
        ).encode(
            x=alt.value(0),  # pixels from left
            y=alt.value(0),  # pixels from top
            text=alt.value('No adverse event recorded!')
        )
    else:
        if len(patientoutcomes[patientoutcomes['value'] != '?']) > 0:
            outcomechart = alt.Chart(patientoutcomes,height=100,width=700).mark_point().encode(
                x=alt.X('value:Q',axis=alt.Axis(grid=False,tickMinStep=1),title=None),
                y=alt.Y('variable',title=['Outcome'])
            )
        if '?' in patientoutcomes['value'].unique():
            agvhd_section = alt.Chart({'values':[{}]},width=700).mark_text(
                align="left", baseline="top",fontStyle='bold',
                    fontSize=12
            ).encode(
                x=alt.value(0),  # pixels from left
                y=alt.value(0),  # pixels from top
                text=alt.value('Acute GvHD recorded without specified timepoint!')
            )
            if outcomechart == None:
                outcomechart = agvhd_section
            else:
                outcomechart &= agvhd_section
    if outcomechart == None:
        outcomechart = alt.Chart({'values':[{}]},width=700).mark_text(
        align="left", baseline="top",fontStyle='bold',
                fontSize=12
        ).encode(
            x=alt.value(0),  # pixels from left
            y=alt.value(0),  # pixels from top
            text=alt.value('Control Group')
        )        
    outcomechart = outcomechart.properties(title='Recorded Outcomes')
    
    #### Outliers
    
    outliertable = patientsub[['Outlier High Unclassified', 'Outlier High Human', 'Outlier High ARG Reads',
                               'Outlier Low Bacteria', 'Outlier High Fungi', 'Outlier High Other Eukaryota',
                               'Outlier High Virus', 'Outlier High Archae', 'Outlier High DNA per g Stool',
                               'Outlier High Total Reads', 'Outlier High Median Reads Length', 'time']].melt(id_vars = 'time')
    outlierchart = alt.Chart(outliertable[outliertable['value']=='y']).mark_point().encode(
        x = 'time',
        y = 'variable')
    
    #### Mikrogramm DNA 
    
    
    dnachart= alt.Chart(patientsub,height=100,width=700).mark_bar(stroke='black',strokeWidth=0.5,strokeOpacity=0.9).encode(
        x=alt.X('time:Q',axis=alt.Axis(grid=False,tickMinStep=1),title=None),
        y=alt.Y('Mikrogramm DNA pro g Stuhl ',title=['DNA','(µg/g Stool)'],scale=alt.Scale(type='log'),axis=alt.Axis(tickCount=10, format=".1e")),
        order='time',
        tooltip=['Mikrogramm DNA pro g Stuhl ']
    )


    libchart = alt.Chart(patientsub,height=100,width=700).mark_bar(stroke='black',strokeWidth=0.5,strokeOpacity=0.9).encode(
        x=alt.X('time:Q',axis=alt.Axis(grid=False,tickMinStep=1),title=None),
        y=alt.Y('Total Reads',title=['Total Reads'],scale=alt.Scale(type='log'),axis=alt.Axis(tickCount=10, format=".1e")),
        order='time',
        tooltip=['Total Reads']
    )

    divchart = alt.Chart(patientsub,height=100,width=700,title='Diversity Analysis based on normalized read fractions').mark_bar(stroke='black',strokeWidth=0.5,strokeOpacity=0.9).encode(
        x=alt.X('time:Q',axis=alt.Axis(grid=False,tickMinStep=1),title=None),
        y=alt.Y('S_True',title=['Species Count','(Normalized, No Human)']),
        order='time'
    )
    
    shannonchart = alt.Chart(shannon_df[shannon_df['sample'].isin(samples)],height=100,width=700).mark_bar(stroke='black',strokeWidth=0.5,strokeOpacity=0.9).encode(
        x=alt.X('time:Q',axis=alt.Axis(grid=False,tickMinStep=1),title=None),
        y=alt.Y('Shannon Quotient',title=['Shannon Quotient'])
        
    )
    
    #### Unclassified, Classified, HUMAN
    Gruppen=[
        'Other classified',
        'Unclassified',
        'Human'
    ]
    z = patientsub[[
        'Other classified_Prozent ohne human pro total reads',
        'Unclassified_Prozent pro total reads',
        'Human_Prozent pro total reads',
        'time',
        'Total Reads']]
    
    z = z.melt(id_vars=['time','Total Reads'])
    
    z[['Group','Rest']] = z['variable'].str.split(
        '_',
        expand=True
    )

    z = z.drop(columns=['variable'])
    z = z.pivot(index=['time','Total Reads'],columns='Group',values='value')

    z = z.reset_index()
    
    z = z.melt(id_vars=['time','Total Reads'])
    z = z.rename(columns={'value' : 'Fraction'})
    
    z['Count'] = z['Fraction'] * z['Total Reads']/100
    
    toplevelreadchart = alt.Chart(z,title='Top-Level Read Counts').transform_calculate(
                      order=f"-indexof({Gruppen}, datum.Group)"
                    ).mark_bar(stroke='black',strokeWidth=0.5,strokeOpacity=0.9).encode(
        x=alt.X("time:Q", title=None, axis=alt.Axis(labels=False), scale=alt.Scale(paddingInner=1)), 
        y=alt.Y("Fraction:Q",title='Fraction [%]',scale=alt.Scale(type='symlog',domain=[0,100]), 
                axis=alt.Axis(grid=False),stack=True), 
        color=alt.Color("Group:N",sort=Gruppen,legend=alt.Legend(title=None,orient='top'),
                    scale=alt.Scale(
                        domain=Gruppen,range=['#EE6677', '#228833', '#CCBB44'])
                       ),
        order='order:Q',
        tooltip=['Fraction','Count']
    )
    
    ###### ARG 
    
    argchart = alt.Chart(patientsub,height=100,width=700,title='AB Resistance Analysis').mark_bar(stroke='black',strokeWidth=0.5,strokeOpacity=0.9).encode(
        x=alt.X('time:Q',axis=alt.Axis(grid=False,tickMinStep=1),title=None),
        y=alt.Y('ARG Reads pro Eingabe Reads *10000',title=['Reads with','Resistancegenehits/10K']),
        order='time'
    )
    
    #### Top Level View
    Gruppen=[
        'Bacteria',
        'Fungi',
        'Viruses'
    ]
    z = patientsub[['Bacteria_Prozent pro classified ohne human',
                    'Fungi_Prozent pro classified ohne human',
                    'Viruses_Prozent pro classified ohne human','time','classified reads ohne human']]
    z = z.melt(id_vars=['time','classified reads ohne human'])
    z[['Group','Rest']] = z['variable'].str.split('_',expand=True)

    z = z.drop(columns=['variable'])
    z = z.pivot(index=['time','classified reads ohne human'],columns='Group',values='value')

    z = z.reset_index()
    z = z.melt(id_vars=['time','classified reads ohne human'])
    z = z.rename(columns={'value' : 'Fraction'})
    z['Count'] = z['Fraction'] * z['classified reads ohne human']/100
    
    toplevelcompchart = alt.Chart(z,title='Top-Level Composition, Normalized, No Human').transform_calculate(
                      order=f"-indexof({Gruppen}, datum.Group)"
                    ).mark_bar(stroke='black',strokeWidth=0.5,strokeOpacity=0.9).encode(
        x=alt.X("time:Q", title=None, axis=alt.Axis(labels=False), scale=alt.Scale(paddingInner=1)), 
        y=alt.Y("Fraction:Q",title='Fraction [%]',scale=alt.Scale(type='symlog',domain=[0,100]), 
                axis=alt.Axis(grid=False),stack=True), 
        color=alt.Color("Group:N",sort=Gruppen,legend=alt.Legend(title=None,orient='top'),
                    scale=alt.Scale(
                        domain=Gruppen,range=['#EE6677', '#228833', '#CCBB44'])
                       ),
        order='order:Q',
        tooltip=['Fraction','Count']
    )

    combinedchart = reduce(lambda x,y : (x&y).resolve_scale(
        x='shared',color='independent'
    ),[
        leukochart,
        phasechart,
        outcomechart,
        outlierchart,
        dnachart,
        libchart,
        toplevelreadchart,
        toplevelcompchart]+
        compcharts
        +mscharts+[
          divchart,
        shannonchart,
        argchart
    ]).configure_axisY(minExtent=40).properties(title='Patient {} / Initial Cluster: {}'.format(patient,cluster))

    combinedchart.save('Output/Patient_Journey/{}.html'.format(patient))
    

# Bray-Curtis Distance based PCoA <a class="anchor" id="pcoa"></a>
$$
B_{i,j} = 1 - \frac{2C_{i,j}}{S_i+S_j}
$$

For relative abundances, this becomes:

$1-C_{i,j}$

In [ ]:
subtable = get_normalized_abundances(kraken_dataframe,
                                        level='G',
                                        samples=PAPER_SAMPLES,
                                        excluded_taxa_filter=[CHORDATA],
                                        included_taxa_filter=None,
                                       normalize=False)



subtable['readAnteil'] = subtable['readcount'] / subtable.groupby(['time','patientid'])['readcount'].transform('sum')

subtable = subtable.drop(columns=['level','readcount'])

pivot_table = subtable.pivot_table(index=['patientid','time'],columns=['taxon'],aggfunc=sum).fillna(0)
data = pivot_table.values

samples = pivot_table.index.tolist()
taxa_count = len(data[0])

def braycurtis(indexA,indexB):
    cij = 0
    for taxonIndex in range(taxa_count):
        cij += min(
            data[indexA][taxonIndex],
            data[indexB][taxonIndex]
        )
    return 1-cij

bc_tuples = []

distance_matrix = np.zeros(shape=(len(samples),len(samples)))

for x in range(len(samples)):
    for y in range(x+1,len(samples)):
        print('Calculating all distancesfor sample {} / {}'.format(x+1,len(samples)),end='\r')
        distance = braycurtis(x,y)
        distance_matrix[x][y] = distance
        distance_matrix[y][x] = distance
        bc_tuples.append((samples[x][0]+'/'+str(samples[x][1]),samples[y][0]+'/'+str(samples[y][1]),distance))
        
INVERT_X = True

PCoA = ordination.pcoa(distance_matrix,number_of_dimensions=2)

tuples = []

for idx,s in PCoA.samples.iterrows():
    sample = samples[int(idx)]
    patient = sample[0]
    time = sample[1]

    tuples.append((s.PC1,s.PC2,patient,time))

pcoa_table = pd.DataFrame(
    tuples,
    columns=['x','y','patient','time']
)

pcoa_table['Name'] = pcoa_table['patient'].astype(str)+'/'+pcoa_table['time'].astype(str)


pcoa_table = pd.merge(sample_statistics,pcoa_table,left_on=['time','PatID'],right_on=['time','patient'],how='right')

pcoa_table['Sample Type'] = pcoa_table['Name'].apply(lambda x : 'Control' if x.startswith('G') else 'patient')

if INVERT_X:
    pcoa_table['x'] = -pcoa_table['x']

selection = alt.selection_multi(fields=['last_sample_reconstitution'])
color = alt.condition(selection,
                      alt.Color('Startcluster:N', legend=None),
                      alt.value('lightgray'))


pcoa_combined = (alt.Chart(pcoa_table).transform_calculate(
    order = f"indexof({selection.name}.Startcluster || [],datum.Startcluster)"
).mark_point(filled=True).encode(
    x='x:Q',
    y='y:Q',
    color=color,
    order=alt.Order('order:N',sort='ascending'),
    tooltip=['Name']
).interactive() | alt.Chart(pcoa_table).mark_rect().encode(
    y=alt.Y('last_sample_reconstitution:N',title=None),
    color=color
).add_selection(selection))

pcoa_combined.save(
    'Output/PCOA.html'
)

pcoa_combined

In [ ]:
bc_frame = pd.DataFrame(bc_tuples,columns=['Sample X','Sample Y','BCD'])
bc_frame[['Patient X','Time X']] = bc_frame['Sample X'].str.split('/',expand=True)
bc_frame[['Patient Y','Time Y']] = bc_frame['Sample Y'].str.split('/',expand=True)

#Determine most extreme timepoints for each patient

extreme_points = pd.concat(
    [bc_frame.groupby('Patient X')['Time X'].min().rename('Earliest Time'),
    bc_frame.groupby('Patient X')['Time X'].max().rename('Latest Time')],axis=1
).reset_index()

#Eliminate rows where earliest is >= 0 or latest is <= 0
eliminated = extreme_points[
    (extreme_points['Earliest Time'].astype(int) < 0)&
    (extreme_points['Latest Time'].astype(int) > 0)
]

print(
    'For the following patients no valid pre/post pair could be generated: {} (They will be EXCLUDED from analysis)'.format(
        
        set(extreme_points['Patient X']).difference(set(eliminated['Patient X']))
    )
)

bc_frame = bc_frame[
    (bc_frame['Patient X'].isin(eliminated['Patient X']))&
    (bc_frame['Patient Y'].isin(eliminated['Patient X']))&
    (bc_frame['Patient X'] == bc_frame['Patient Y'])
]

eliminated = eliminated.set_index('Patient X')

bc_frame['Earliest X'] = bc_frame['Patient X'].apply(lambda x :  eliminated.loc[str(x)]['Earliest Time'])
bc_frame['Latest X'] = bc_frame['Patient X'].apply(lambda x :  eliminated.loc[str(x)]['Latest Time'])

bc_frame = bc_frame[
    (bc_frame['Time X'] == bc_frame['Earliest X'])&
    (bc_frame['Time Y'] == bc_frame['Latest X'])

]

bc_frame['Time X'] = bc_frame['Time X'].astype(int)
bc_frame['Time Y'] = bc_frame['Time Y'].astype(int)

bc_frame = pd.merge(
    bc_frame,
    sample_statistics[['PatID','time','Startcluster']],
    how='left',
    left_on=['Patient X','Time X'],
    right_on=['PatID','time']
)

bc_frame

bc_frame = pd.merge(
    bc_frame,
    sample_statistics[['PatID','time','leukocytephase_cluster_2_kurz']],
    how='left',
    left_on=['Patient Y','Time Y'],
    right_on=['PatID','time']
)[['Sample X','Sample Y','BCD','Startcluster','leukocytephase_cluster_2_kurz']]
alt.Chart(bc_frame).mark_boxplot().encode(
    x='Startcluster:N',
    y='BCD'
)


# Sampling Time Overview <a class="anchor" id="sampletimes"></a>

In [ ]:
outcomes_reduced = outcomes[
    ['Pat ID',
     'Day relative to HSCT',
     'Day relative to HSCT.1',
     'Day relative to HSCT.2',
     'Day relative to HSCT.3']
         ].rename(
    columns={
        'Day relative to HSCT' : 'Relapse',
        'Day relative to HSCT.1' : 'Acute GvHD Grade 1-2',
        'Day relative to HSCT.2' : 'Acute GvHD Grade 3-4',
        'Day relative to HSCT.3' : 'Death',
    }
)
outcomes_reduced['No Adverse Event']=outcomes_reduced.apply( lambda x : 0 if x.count() <= 1 else np.NaN,axis=1)
outcomes_reduced

outcomes_reduced = outcomes_reduced.melt(id_vars=['Pat ID'])
outcomes_reduced = outcomes_reduced[(outcomes_reduced['value']==outcomes_reduced['value'])]

outcomes_reduced.loc[
    (outcomes_reduced['Pat ID']=='18.2'),'KI ID'
] = '18.2'
outcomes_reduced.loc[(outcomes_reduced['Pat ID']=='18.2'),'value'] -= OFFSET_PAT_18
outcomes_reduced = outcomes_reduced[outcomes_reduced['variable'] != 'No Adverse Event']

outcomes_reduced = outcomes_reduced.rename(columns={
    'Pat ID' : 'PatID',
    'variable' : 'Adverse Event',
    'value' : 'time'
})


############

sample_statistics['id'] = sample_statistics['PatID']+'/'+sample_statistics['time'].astype(str)
overview = sample_statistics[sample_statistics['id'].isin(PAPER_SAMPLES)]
overview = overview[~overview['PatID'].str.startswith('G')]


combined = pd.concat([overview,outcomes_reduced])



patientlist_sorted = []

for startcluster in combined['Startcluster'].unique():
    clusterlist = sorted(combined[combined['Startcluster'] == startcluster]['PatID'].unique())
    patientlist_sorted += clusterlist

        
overview = (alt.Chart(combined[combined['Adverse Event'] == combined['Adverse Event']]).mark_point(size=38,color='black').encode(
    x=alt.X('time',scale=alt.Scale(type='symlog'),axis=alt.Axis(grid=False)),
    y=alt.Y('PatID',sort=patientlist_sorted),
    shape=alt.Shape('Adverse Event:N',scale=alt.Scale(
        domain=['Acute GvHD Grade 1','Acute GvHD Grade 3','Death','Relapse'],
        range=['circle','square','cross','triangle']))
)+alt.Chart(combined,width=800).mark_circle().encode(
    x=alt.X('time',scale=alt.Scale(type='symlog'),axis=alt.Axis(grid=False)),
    y=alt.Y('PatID:N',sort=patientlist_sorted,axis=alt.Axis(grid=True)),
    color='Startcluster:N'
)+alt.Chart(combined).mark_text(dx=0,dy=-6).encode(
    x=alt.X('time',scale=alt.Scale(type='symlog'),axis=alt.Axis(grid=False)),
    y=alt.Y('PatID:N',sort=patientlist_sorted,axis=alt.Axis(grid=True)),
    text='time'
))

overview.save('Output/Samples.html')

overview

# Zymo Std. Analysis <a class="anchor" id="zymo"></a>

In [ ]:
species_level = get_normalized_abundances(kraken_dataframe,samples=['Zymo_Power/-100'],level='S',excluded_taxa_filter=None,normalize=False)

species_level['Read Fraction (%)'] = species_level['readcount'] / species_level.groupby(['sample'])['readcount'].transform('sum')
species_level = species_level.rename(columns={'sample' : 'Sample ID','taxon' : 'Taxon'})
species_level = pd.concat([species_level,zymo_theory])[['Sample ID','Taxon','Read Fraction (%)']]


melt = zymo_minimap.melt(id_vars='Index')

def clean(x):
    x = x.replace('_',' ').replace('albican','albicans')  
    if x.startswith('Escherichia coli'):
        return 'Escherichia coli'
    return x

melt['variable'] = melt['variable'].apply(
    clean
)


melt = melt.rename(
    columns={
    'variable' : 'Taxon',
        'Index' : 'Sample ID',
        'value' : 'Read Fraction (%)'
    }
)


melt = melt[melt['Sample ID'].isin(['Power/Reads','Power/Bases'])]


melt['Sample ID'] = melt['Sample ID'].apply(lambda x : x + '_Minimap2')

melt['Taxon'] = melt['Taxon'].apply(lambda x : 'Unmapped' if x == 'unmapped' else x)


combined = pd.concat([melt,species_level])


taxa_we_look_at = list(combined.groupby('Taxon')['Read Fraction (%)'].mean().sort_values(ascending=False)[:(20+1)].keys())
print('Determined the following taxa as relevant:',taxa_we_look_at)

#assign everything else to the "other" group and readjust sum
combined.loc[~combined['Taxon'].isin(taxa_we_look_at), 'Taxon'] = 'Other'
combined = combined.groupby(['Taxon','Sample ID'],as_index=False).sum()


colorMap = {}

taxa = combined['Taxon'].unique()

palette = sns.color_palette("tab20",n_colors=len(taxa)) #hls

for tax,col in zip(list(taxa),palette):
    colorMap[tax] = col #colors.to_hex(col)
    
altdomain = []
altrange = []

for x in combined['Taxon'].unique():
    
    if x == 'Other' or x == 'Unassigned at Level' or x == 'Unmapped':
        continue

    c = colorMap[x]
    altdomain.append(x)
    color = colors.to_hex(c)
    altrange.append(color)
    
altdomain.append('Other')
altrange.append(colors.to_hex((1,1,1)))
altdomain.append('Unassigned at Level')
altrange.append(colors.to_hex((0.15,0.15,0.15)))
altdomain.append('Unmapped')
altrange.append(colors.to_hex((0.45,0.45,0.45)))

c= alt.Chart(
  combined
).transform_calculate(
order=f"-indexof({altdomain}, datum.Taxon)"
).mark_bar(stroke='black',strokeWidth=0.5,strokeOpacity=0.9).encode(
    x=alt.X('Sample ID:N',sort=['Zymo_Power/-100']
),
    y=alt.Y('Read Fraction (%):Q',scale=alt.Scale(domain=(0,1))),
    color=alt.Color('Taxon:N',legend=alt.Legend(columns=2,symbolLimit=0,labelLimit=0),scale=alt.Scale(domain=altdomain,range=altrange)),
    tooltip=['Read Fraction (%)','Taxon'],
    order=alt.Order('order:Q')
)
c.save('Output/Zymo_Overview.html')
c

# Eukaryotes  <a class="anchor" id="eukaryotes"></a>

In [ ]:
data = pd.read_csv('Input/eukaryotes.csv',dtype={'Taxon' : str})
data['Taxon Name'] = data['Taxon'].map(idtonames)
data['Chi Average'] = data['Chi Values'].apply(lambda x : sum(int(y) for y in eval(x))/len(eval(x)) if len(eval(x)) != 0 else '?')


taxa_sorting = data.groupby('Taxon')['Total Reads'].sum().sort_values(ascending=False).keys().to_list()

tooltip = ['Mapped Reads','Horizontal Coverage','Taxon Name','Sample','Chi Average','Total Reads']


alt.Chart(data,title='Horizontal Coverage').mark_rect().encode(
    x='Sample:N',
    y=alt.Y('Taxon Name:N',sort=taxa_sorting),
    color=alt.condition(
        alt.datum['Horizontal Coverage'] == 0,
        alt.value('lightgrey'),
        'Horizontal Coverage:Q'
    ),
    tooltip=tooltip
    
).save('Output/HorizontalCoverageEukaryota.html')



data['Mapped Reads Count'] = data['Mapped Reads'] * data['Total Reads']

top_level_stats = pd.concat(
    [data.groupby('Taxon Name')['Total Reads'].sum(),
    data.groupby('Taxon Name')['Mapped Reads Count'].sum()/data.groupby('Taxon Name')['Total Reads'].sum()],axis=1
).reset_index().rename(columns={0 : 'Mapped Fraction'}).sort_values(by='Mapped Fraction')

top_level_stats.to_csv('Output/excel_eukaryota.csv',index=False)

data.groupby('Taxon Name')[['Mapped Reads Count','Total Reads']].sum().sort_values(by='Total Reads').to_csv('Overview_Eukaryota.csv')



aggregated = data.groupby('Taxon Name',as_index=False)[['Total Reads','Mapped Reads Count']].sum()
aggregated['Fraction'] = aggregated['Mapped Reads Count'] / aggregated['Total Reads']

alt.Chart(aggregated,width=800,height=800).mark_point().encode(
    x='Total Reads',
    y='Fraction',
    color='Taxon Name',
    tooltip=['Total Reads','Mapped Reads Count','Taxon Name','Fraction']
).interactive().save('Output/VerifiedEukaryotaVSTotalReads.html')

In [ ]:
x = 20

found_in_x_samples = data.groupby('Taxon Name')['Sample'].size()
found_in_x_samples = found_in_x_samples[found_in_x_samples > x]


In [ ]:
x = 500

average_above_x = data.groupby('Taxon Name')['Total Reads'].mean()
average_above_x = average_above_x[average_above_x > x]


In [ ]:
subset = data[
    data['Taxon Name'].isin(
        set(found_in_x_samples.keys()).intersection(set(average_above_x.keys()))
    )
   
]

len(subset['Taxon Name'].unique())

In [ ]:

alt.Chart(subset).mark_boxplot().encode(
    x=alt.X('Taxon Name'),
    y='Mapped Reads'
).save('Output/VerificationEukaryota.html')

## Check Metamaps Kongruence

In [ ]:
metamaps_dataframe['Sample'] = metamaps_dataframe['patientid']+'_'+metamaps_dataframe['time'].astype(str)
merged = pd.merge(data,metamaps_dataframe[
    (metamaps_dataframe['Sample'].isin(data['Sample']))&
    (metamaps_dataframe['readcount'] >= 100)
]
                  
                  ,left_on=['Sample','Taxon Name'],right_on=['Sample','taxon'],how='left')
merged = merged.fillna(0)
merged['Excess Kraken'] = merged['Total Reads']-merged['readcount']


In [ ]:
alt.Chart(merged).mark_point().encode(
    y=alt.Y('Excess Kraken',scale=alt.Scale(type='symlog')),
    x=alt.X('Mapped Reads',bin=alt.Bin(maxbins=10))
).save('Output/ExcessKrakenEukaryota.html')

# Crassphage <a class="anchor" id="crassphage"></a>

In [ ]:
kraken_data = kraken_dataframe[kraken_dataframe['taxon'].isin(['uncultured crAssphage','root'])]
kraken_data = kraken_data[kraken_data['sample'].isin(PAPER_SAMPLES)]
kraken_data = kraken_data.pivot(
    index='sample',columns='taxon',values='readcount'
).fillna(0)

kraken_data['crassphage_detected'] = kraken_data['uncultured crAssphage']/kraken_data['root']

kraken_data = kraken_data.reset_index()

def rename(x):
    split = x.rsplit('_',1)
    return split[0]+'/'+split[1]

minimap_data = pd.read_csv('Input/Crassphage/summary.csv')
minimap_data['Sample'] = minimap_data['Sample'].apply(rename)


minimap_data_aggregated = minimap_data.groupby('Sample')[['Fraction Mapped','Mapped Reads']].sum().reset_index()


combined = pd.merge(kraken_data,minimap_data,left_on='sample',right_on='Sample',how='left')

max_value = max(
    combined['Fraction Mapped'].max(),combined['crassphage_detected'].max()
)

line = pd.DataFrame({
    'X': [0, max_value],
    'Y': [0, max_value],
})


(alt.Chart(combined,width=700,height=700).mark_point().encode(
    y='sum(Fraction Mapped)',
    x='crassphage_detected',
    tooltip='sample'
)+alt.Chart(line,width=700,height=700).mark_line(color= 'lightgray').encode(
        x= 'X',
        y= 'Y'
    )).interactive().save('Output/CrassphageKrakenVsMinimap.html')

## Heatmap, which Crassphage

In [ ]:
heatmap_table = minimap_data.pivot(index='Sample',columns='Reference',values='Fraction Mapped').fillna(0).melt(ignore_index=False).reset_index().rename(columns={'value' : 'Fraction Mapped'})
heatmap_table['Unambiguous'] = heatmap_table['Reference'] != 'Ambiguous'
heatmap_table_simplified = heatmap_table.groupby(['Sample','Unambiguous'],as_index=False)['Fraction Mapped'].sum()

top_x = 100


heatmap_table_filtered = heatmap_table[
    (heatmap_table['Reference']!='Ambiguous')&
    (heatmap_table['Sample'].isin(PAPER_SAMPLES)  )  
]
heatmap_table_filtered['Fraction Mapped']=heatmap_table_filtered['Fraction Mapped']/heatmap_table_filtered.groupby('Sample')['Fraction Mapped'].transform('sum')
top_refs = heatmap_table_filtered.groupby('Reference')['Fraction Mapped'].sum().sort_values(ascending=False).keys().tolist()[:top_x]

heatmap_table_filtered = heatmap_table_filtered[
    
    heatmap_table_filtered['Reference'].isin(top_refs)
]

In [ ]:
(alt.Chart(heatmap_table_simplified[heatmap_table_simplified['Sample'].isin(PAPER_SAMPLES)]).mark_bar().encode(
    x=alt.X('Sample',sort=sort_samples(PAPER_SAMPLES),axis=alt.Axis(orient='top')),
    y=alt.Y('Fraction Mapped',stack=True),
    color='Unambiguous'
)&alt.Chart(heatmap_table_filtered).mark_rect().encode(
    x=alt.X('Sample',sort=sort_samples(PAPER_SAMPLES),axis=None),
    y=alt.Y('Reference',sort=top_refs),
    color=alt.condition(
        alt.datum['Fraction Mapped'] == 0,
        alt.value('lightgrey'),
        alt.Color('Fraction Mapped:Q',title='Fraction of uniquely assigned reads')
    )
)).resolve_scale(x='shared',color='independent').save('Output/CrassphageOverview.html')

## Migration Analysis

In [ ]:
migration = pd.read_csv('Input/Crassphage/migration.csv',dtype={'Tax ID' : str})
migration['Fraction'] = migration['Reads']/migration.groupby('Sample')['Reads'].transform('sum')
migration['Taxon Name'] = migration['Tax ID'].map(idtonames)

top_hits = migration.groupby('Tax ID')['Fraction'].mean().sort_values(ascending=False).keys()[:20]
migration = migration[migration['Tax ID'].isin(top_hits)]


alt.Chart(migration).mark_boxplot().encode(
    y='Fraction',
    x=alt.X('Taxon Name:N'),
    tooltip=['Reads','Sample','Fraction','Taxon Name']
)

In [ ]:
details = pd.read_csv('Input/crassphage_details.csv')

charts = []

for metric in ['Percentage Aligned','Mapping Quality','Identity']:

    amount,edges = np.histogram(details[metric],bins=100)

    centers = []
    for x,y in zip(edges[:-1],edges[1:]):
        centers.append((x+y)/2)

    tuples = []
    for x,y in zip(centers,amount):
        tuples.append((x,y))

    df = pd.DataFrame(tuples,columns=[metric,'Count'])

    c = alt.Chart(df).mark_bar().encode(
        x=metric,
        y=alt.Y('Count',scale=alt.Scale(type='symlog'))
    )
    
    charts.append(c)
    
reduce(lambda x,y : x&y, charts).save('Output/CrassphageAlignmentDetails.html')

# Marker Species Overview

In [ ]:
print('Load Marker Gene Data')

ms_frame = get_normalized_abundances(kraken_dataframe,normalize=True,level='S',samples=PAPER_SAMPLES)

ms_frame=ms_frame[ms_frame['sample'].isin(PAPER_SAMPLES)] 
ms_frame['estimated abundance'] = (ms_frame['readcount'] / ms_frame.groupby(['time','patientid'])['readcount'].transform('sum')).round(10)

ms_frame = ms_frame[
    ms_frame['taxon'].str.startswith(tuple(MARKER_SPECIES_REDUCED))
]

ms_frame = ms_frame[ms_frame['taxon'] != 'Saccharomyces cerevisiae x Saccharomyces kudriavzevii']

ms_frame = ms_frame.pivot(index=['patientid','time'],
                          columns=['taxon'],
                          values=['estimated abundance']).fillna(0).reset_index().melt(id_vars=['patientid','time'])


ms_frame =pd.merge(
    ms_frame,sample_statistics[['PatID','time','leukocytephase_cluster_2_kurz']],
    how='left',
    left_on=['patientid','time'],
    right_on=['PatID','time']
)[['patientid','time','taxon','value','leukocytephase_cluster_2_kurz']]

In [ ]:
alt.Chart(ms_frame,width=100,height=100).mark_boxplot(ticks=True,median={'color':'black'}).encode(
    column=alt.Column(
        'leukocytephase_cluster_2_kurz:O',
        spacing=0,
        sort=['Healthy', 'Pre TX','Leukozytopenia','Reconstitution'],
        title=None
    ),
    y=alt.Y('value:Q',axis=alt.Axis(grid=False,minExtent=40), title='Estimated Abundance'),
    color='taxon',
    x=alt.X('taxon',title=None)
)

# Illumina Sequencing / Population Shifts <a class="anchor" id="strains"></a>

## Load samplesheet (To resolve illumina ids -> patient/time)

In [ ]:
samplesheetDictPatient = {}
samplesheetDictTime = {}
    
samplesheet = pd.read_csv('Input/samples.tsv',sep='\t')
#Retain only entries that have illumina files
samplesheet=samplesheet[samplesheet['illuminafile'] ==samplesheet['illuminafile']]
for idx,row in samplesheet.iterrows():
    samplesheetDictPatient[row['illuminafile']] = row['patientid']
    samplesheetDictTime[row['illuminafile']] = row['time']


## Load and process/annotate distances

In [ ]:
HIGH_CONFIDENCE_CUTOFF = 50
output_folder = 'Output/StrainAnalysis/GutTrSnp_{}'.format(HIGH_CONFIDENCE_CUTOFF)

guttrsnp_distances = pd.read_csv('Input/GutTrSnp/RealData_{}/distances.csv'.format(
   HIGH_CONFIDENCE_CUTOFF 
),dtype={
    'Destination ID':str,
    'Source ID' :str,
    'Taxon' : str
})

#Kick out the stuff where there was no overlap and thus no distance
guttrsnp_distances = guttrsnp_distances.dropna()
guttrsnp_distances=guttrsnp_distances[guttrsnp_distances['Share Of Overlap'] >= 0.01]

guttrsnp_distances['Taxon Name'] = guttrsnp_distances['Taxon'].map(idtonames)
guttrsnp_distances['Source'] = guttrsnp_distances['Source ID']+'/'+guttrsnp_distances['Source Time'].astype(str)
guttrsnp_distances['Destination'] = guttrsnp_distances['Destination ID']+'/'+guttrsnp_distances['Destination Time'].astype(str)
guttrsnp_distances['PrePost Pair'] = (
    guttrsnp_distances['Source ID'] == guttrsnp_distances['Destination ID']
)&(
guttrsnp_distances['Source Time'] < 0
)&(
guttrsnp_distances['Destination Time'] > 0
)
guttrsnp_distances['Within Patient'] = guttrsnp_distances['Source ID'] == guttrsnp_distances['Destination ID']
#DoT
guttrsnp_distances['Elapsed Time'] = guttrsnp_distances['Destination Time'] - guttrsnp_distances['Source Time']
guttrsnp_distances['Distance over Time'] = guttrsnp_distances['GutTrSnp Distance'] / guttrsnp_distances['Elapsed Time']

guttrsnp_distances['Distance Name'] = guttrsnp_distances['Source Time'].astype(str) + '->' + guttrsnp_distances['Destination Time'].astype(str)

coverages = pd.read_csv('Input/GutTrSnp/RealData_{}/coverages.csv'.format(
    HIGH_CONFIDENCE_CUTOFF
),dtype={
    'Patient ID' : str,
    'Taxon ID' : str
})
guttrsnp_distances=pd.merge(guttrsnp_distances,coverages,how='left',left_on=['Source ID','Source Time','Taxon'],right_on=['Patient ID','Time','Taxon ID'])
guttrsnp_distances=pd.merge(guttrsnp_distances,coverages,how='left',left_on=['Destination ID','Destination Time','Taxon'],right_on=['Patient ID','Time','Taxon ID'])



guttrsnp_distances = guttrsnp_distances.drop(columns=[
    'Patient ID_x','Patient ID_y',
    'Time_x','Time_y',
    'Taxon ID_x','Taxon ID_y'
])


bins=[0,10,20,50,100,200,500,1000,2000]


guttrsnp_distances['VCG Source'] = pd.cut(guttrsnp_distances['Average Vertical Coverage_x'],bins)
guttrsnp_distances['VCG Destination'] = pd.cut(guttrsnp_distances['Average Vertical Coverage_y'],bins)
guttrsnp_distances = guttrsnp_distances.dropna()
guttrsnp_distances['VCG Source']=guttrsnp_distances['VCG Source'].astype(str)
guttrsnp_distances['VCG Destination']=guttrsnp_distances['VCG Destination'].astype(str)

timepoints = {}

for taxon in guttrsnp_distances['Taxon'].unique():
    for source_id in guttrsnp_distances['Source ID'].unique():
        subtable = guttrsnp_distances[
            (
                guttrsnp_distances['Taxon'] == taxon
            )&(
                guttrsnp_distances['Source ID'] == source_id
                
            )
        ]
        timepoints[(taxon,source_id)] = sorted(subtable['Source Time'].unique().tolist())
        
def sequential_test(row):
    if row['Within Patient']:
        if (row['Taxon'],row['Source ID']) in timepoints:
            sequence = timepoints[(row['Taxon'],row['Source ID'])]
            if sequence.index(row['Destination Time'])-sequence.index(row['Source Time']) == 1:
                return True
    return False

guttrsnp_distances['Sequential'] = guttrsnp_distances.apply(sequential_test,axis=1)

In [ ]:
mapping_to_simulation={
    'Bacteroides vulgatus ATCC 8482' : 'PVulgatus',
    'Enterococcus faecium' : 'EFaecium',
    'Lactobacillus gasseri ATCC 33323 = JCM 1131' : 'LGasseri',
    'Shigella sonnei 53G' : 'EColi',
    
}

guttrsnp_distances_simulation = pd.read_csv('Input/GutTrSnp/SubspeciesSimulation/aggregatedDistances.csv',dtype={'Taxon' : str}).dropna()


In [ ]:
# Sanity Checks / Generic Analysis


os.makedirs(output_folder,exist_ok=True)

plots = []
curation = []
for taxon in guttrsnp_distances['Taxon Name'].unique():#['Bacteroides vulgatus ATCC 8482','Flavonifractor plautii','Bacteroides uniformis','Parabacteroides distasonis ATCC 8503','Parabacteroides merdae']:
    
    taxontable = guttrsnp_distances[
        (guttrsnp_distances['Taxon Name'] == taxon)&
        (guttrsnp_distances['Sequential'])&
        (guttrsnp_distances['Share Of Overlap'] >= 0.5)
    ]
       
    taxontable['PrePost Pair'] = taxontable['PrePost Pair'].map(
        {
            True : 'Yes',
            False : 'No'
        }
    )
   
    if len(taxontable) < 3:
        continue    
 
    min_dist = taxontable['GutTrSnp Distance'].min()
    max_dist = taxontable['GutTrSnp Distance'].max()
    step = (max_dist-min_dist)/50
    
    max_line = alt.Chart(
        pd.DataFrame(
        [(max_dist)],columns=['Distance']
    )
    ).mark_rule().encode(
        x='Distance',
        size=alt.value(3)
    )
    
    if taxon in mapping_to_simulation:
        
        subdata = guttrsnp_distances_simulation[
            (guttrsnp_distances_simulation['Simulated Taxon'] == mapping_to_simulation[taxon])&
            (guttrsnp_distances_simulation['Taxon'] == taxontable['Taxon'].values[0])
            
        ]
        
        min_dist = min(subdata['GutTrSnp Distance'].min(),min_dist)
        max_dist = min(subdata['GutTrSnp Distance'].max(),max_dist)
        step = (max_dist-min_dist)/50 
             
    
    plot = alt.Chart(taxontable,height=300,title='Sequential Samples Same Patients').mark_bar().encode(
            x=alt.X('GutTrSnp Distance',bin=alt.Bin(extent=[min_dist,max_dist],step=step),title='Distance'),
            y=alt.Y('count()',axis=alt.Axis(tickMinStep=1),stack=True),
            color=alt.Color('PrePost Pair',title='Across Transplantation')
        )
    
        
    plot = (plot |alt.Chart(taxontable,height=300,title='Sequential Samples Same Patients').mark_point().encode(
            y=alt.Y('GutTrSnp Distance',title='Distance'),
            x='Elapsed Time',
            tooltip=['Source','Destination','Share Of Overlap'],
            color=alt.Color('PrePost Pair',title='Across Transplantation')
        )          
    )
    
    curation.append(
        taxontable
    )
                   
                
    prepairs = guttrsnp_distances[
        (guttrsnp_distances['Taxon Name'] == taxon)&
        (guttrsnp_distances['Source ID'] != guttrsnp_distances['Destination ID'])&
        (guttrsnp_distances['Source Time'] < 0)&
        (guttrsnp_distances['Destination Time'] < 0)
    ]
    
        
    plot =  (plot | alt.Chart(
        prepairs,height=300,title='Pre-Samples Different Patients'
    ).mark_bar().encode(
        x=alt.X('GutTrSnp Distance',bin=alt.Bin(extent=[min_dist,max_dist],step=step),title='Distance'),
            y=alt.Y('count()',axis=alt.Axis(tickMinStep=1))
    ) 
    )
    
    all_distances = guttrsnp_distances[
        (guttrsnp_distances['Taxon Name'] == taxon)&
        (guttrsnp_distances['Share Of Overlap'] >= 0.5)
    ]
    
    plot =  (plot | (alt.Chart(
            all_distances,height=300,title='All Distances'
        ).mark_bar().encode(
            x=alt.X('GutTrSnp Distance',bin=alt.Bin(maxbins=50),title='Distance'),
            y=alt.Y('count()',axis=alt.Axis(tickMinStep=1))
        ) +max_line)
    )

    if taxon in mapping_to_simulation:
        
        subdata = guttrsnp_distances_simulation[
            (guttrsnp_distances_simulation['Simulated Taxon'] == mapping_to_simulation[taxon])&
            (guttrsnp_distances_simulation['Taxon'] == taxontable['Taxon'].values[0])
            
        ]
        
        plot = (plot |(alt.Chart(
            subdata[subdata['GutTrSnp Distance'] != 0],height=300,title='Simulated Data'
        ).mark_bar().encode(
            x=alt.X('GutTrSnp Distance',bin=alt.Bin(maxbins=50),title='Distance'),
            y=alt.Y('count()',axis=alt.Axis(tickMinStep=1))
        )+max_line))
        
    plots.append(
        plot.properties(title=taxon)
    )


          
chart = reduce(lambda x,y : x& y,plots)

chart.save(output_folder+'/Overview.html')

chart

## Visualization of manually curated shifts

In [ ]:
outcomes = pd.read_excel('Input/Annotations/Patient_Statistics (2).xlsx',dtype={'Pat ID' : str})
outcomes_reduced = outcomes[
    ['Pat ID',
     'Day relative to HSCT',
     'Day relative to HSCT.1',
     'Day relative to HSCT.2',
     'Day relative to HSCT.3']
         ].rename(
    columns={
        'Day relative to HSCT' : 'Relapse',
        'Day relative to HSCT.1' : 'Acute GvHD Grade 1-2',
        'Day relative to HSCT.2' : 'Acute GvHD Grade 3-4',
        'Day relative to HSCT.3' : 'Death',
    }
)
outcomes_reduced

outcomes_reduced = outcomes_reduced.melt(id_vars=['Pat ID'])
outcomes_reduced = outcomes_reduced[(outcomes_reduced['value']==outcomes_reduced['value'])]

outcomes_reduced.loc[
    (outcomes_reduced['Pat ID']=='18.2'),'KI ID'
] = '18.2'
outcomes_reduced.loc[(outcomes_reduced['Pat ID']=='18.2'),'value'] -= OFFSET_PAT_18
outcomes_reduced = outcomes_reduced[['Pat ID','variable','value']]
outcomes_reduced = outcomes_reduced[outcomes_reduced['value'] != '?']

outcomes_reduced

manual_curation = pd.read_csv('Input/curation_reduced.csv')
manual_curation['Distance Name'] = manual_curation['Taxon Name']+':'+manual_curation['Source']+'->'+manual_curation['Destination']
manual_curation['Time X'] = manual_curation['Source'].str.split('/',expand=True)[1].astype(int)
manual_curation['Time Y'] = manual_curation['Destination'].str.split('/',expand=True)[1].astype(int)
manual_curation['Pat ID'] = manual_curation['Source'].str.split('/',expand=True)[0]
manual_curation['Any Annotation'] = manual_curation['Clinical Annotation']!='No events tracked'

sample_times = (manual_curation.groupby(['Pat ID','Taxon Name'])['Time X'].apply(list)+manual_curation.groupby(['Pat ID','Taxon Name'])['Time Y'].apply(list)
).reset_index().explode(0).rename(columns={0:'Time'}).drop_duplicates()

charts=[]
for taxon in manual_curation['Taxon Name'].unique():
    st = manual_curation[manual_curation['Taxon Name'] == taxon]
    orm = outcomes_reduced[outcomes_reduced['Pat ID'].isin(st['Pat ID'].unique())]
    orm['variable'] = orm['variable'].replace(
        {'Acute GvHD Grade 3-4' : 'GvHD Grade 3-4'}
    )
    c = alt.Chart(st).mark_line().encode(
        x=alt.X('Time X',title=None),
        x2=alt.X2('Time Y',title=None),
        y=alt.Y('Pat ID',title='Patient'),
        color=alt.Color('Manual Curation',scale=alt.Scale(
            domain=['Shift','Stable'],range=['Orange','Grey']
        ))
    )+alt.Chart(orm).mark_point(color='black',size=56).encode(
        x=alt.X('value',title='Day'),
        y=alt.Y('Pat ID',title='Patient'),
        shape=alt.Shape('variable',
                       scale=alt.Scale(
                           domain=['GvHD Grade 3-4','Relapse','Death'],range=['circle','square','cross']))
    )+alt.Chart(pd.DataFrame([(0)],columns=['x'])).mark_rule().encode(x='x')+alt.Chart(
    sample_times[sample_times['Taxon Name'] == taxon]

    ).mark_point(color='black',filled=True).encode(
        y='Pat ID',
        x=alt.X('Time',title=None)
    )
    charts.append(c.properties(title=taxon,width=800))
reduce(lambda x,y : x&y, charts).resolve_scale(x='shared')